In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision import transforms
from collections import Counter
from torchmetrics import AUROC
from statistics import mean, stdev

In [2]:
transforms = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05, brightness = .05),
    torchvision.transforms.RandomHorizontalFlip(p = 0.5),
    torchvision.transforms.RandomVerticalFlip(p = 0.5),
    torchvision.transforms.RandomRotation(degrees = (0, 180)),
    ])

In [3]:
train_data = torchvision.datasets.ImageFolder(root = '/mnt/d/MHIST/train', transform = transforms )
test_data = torchvision.datasets.ImageFolder(root = '/mnt/d/MHIST/test', transform = transforms)

In [4]:
train_size = int(0.9 * len(train_data))
val_size = len(train_data) - train_size

In [5]:
train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_size, val_size])

In [6]:
train_dataset.dataset.class_to_idx
print(dict(Counter(train_dataset.dataset.targets)))

{0: 1545, 1: 630}


In [7]:
batch_size = 32

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size= batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= batch_size)

In [9]:
model = torchvision.models.resnet18(pretrained = True)
model.fc = nn.Sequential(nn.Linear(512,2))
model = model.to(device = 'cuda')

In [10]:
loss = torch.nn.CrossEntropyLoss()

In [11]:
def train(num_epochs):
    best_accuracy = 0.0
    
    print("Starting training...")
    
    for epoch in range(1, num_epochs + 1):
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_val_loss = 0.0 
        total = 0.0        
        # Training loop
        
        train_auroc = AUROC(pos_label = 1, num_classes = 2)
        val_auroc = AUROC(pos_label = 1, num_classes = 2)
        
        for data in train_loader:
            inputs, outputs = data
            inputs = inputs.to(device = 'cuda')
            outputs = outputs.to(device = 'cuda')
            optimizer.zero_grad()             
            predicted_outputs = model(inputs)   
            train_loss = loss(predicted_outputs, outputs)   
            train_loss.backward()   
            optimizer.step()        
            running_train_loss +=train_loss.item()
            train_auroc_batch = train_auroc(predicted_outputs, outputs)
                        
        # Calculate training loss value 
        train_loss_value = running_train_loss/len(train_loader) 
        avg_train_auroc = train_auroc.compute()
        
        # Validation Loop 
        with torch.no_grad(): 
            model.eval()
            for data in val_loader:
                inputs, outputs = data
                inputs = inputs.to(device = 'cuda')
                outputs = outputs.to(device = 'cuda')
                predicted_outputs = model(inputs)
                val_loss = loss(predicted_outputs, outputs)
                
                _, predicted = torch.max(predicted_outputs, 1)
                running_val_loss += val_loss.item()
                total += outputs.size(0)
                running_accuracy += (predicted == outputs).sum().item()
                val_auroc_batch = val_auroc(predicted_outputs, outputs)
         
                
        # Calculate validation loss value 
        val_loss_value = running_val_loss/len(val_loader) 
        avg_val_auroc = val_auroc.compute()        
        # Calculate accuracy as the number of correct pred
        accuracy = (100 * running_accuracy / total)     
        
        print('EPOCH', epoch,
              'Training Loss: %.4f' %train_loss_value, 
              'Train AUROC: %.4f' %avg_train_auroc,
              'Validation Loss: %.4f' %val_loss_value,
              'Validation AUROC: %.4f' %avg_val_auroc,
              'Accuracy %d %%' % (accuracy))
        train_auroc.reset()
        val_auroc.reset()

In [12]:
def test(model):
    test_auroc = AUROC(pos_label = 1, num_classes = 2)
    with torch.no_grad():
        model.eval()
        for data in test_loader:
            inputs, outputs = data
            inputs = inputs.to(device = 'cuda')
            outputs = outputs.to(device = 'cuda')
            predicted_outputs = model(inputs)
            _, predicted = torch.max(predicted_outputs, 1)
            test_auroc_batch = test_auroc(predicted_outputs, outputs)
    
    avg_test_auroc = test_auroc.compute()
    return avg_test_auroc

In [ ]:
results = []

for i in range(9):
    
    seed = i
    torch.manual_seed(seed)
    
    model = torchvision.models.resnet18(pretrained = True)
    model.fc = nn.Sequential(nn.Linear(512,2))
    model = model.to(device = 'cuda')
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 0.01)
    train(30)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.9, weight_decay = 0.01)
    train(10)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001, momentum = 0.7, weight_decay = 0.01)
    train(10)
    
    results.append(test(model))

Starting training...


/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Argument `pos_label` should be `None` when running multiclass precision recall curve. Got 1
  warnings.warn(*args, **kwargs)


EPOCH 1 Training Loss: 0.5089 Train AUROC: 0.7223 Validation Loss: 0.4567 Validation AUROC: 0.8341 Accuracy 77 %
EPOCH 2 Training Loss: 0.5645 Train AUROC: 0.6668 Validation Loss: 0.4820 Validation AUROC: 0.7836 Accuracy 77 %
EPOCH 3 Training Loss: 0.5064 Train AUROC: 0.7698 Validation Loss: 0.5152 Validation AUROC: 0.7905 Accuracy 72 %
EPOCH 4 Training Loss: 0.4727 Train AUROC: 0.8036 Validation Loss: 0.4748 Validation AUROC: 0.8309 Accuracy 76 %
EPOCH 5 Training Loss: 0.4433 Train AUROC: 0.8294 Validation Loss: 0.4044 Validation AUROC: 0.8516 Accuracy 79 %
EPOCH 6 Training Loss: 0.4229 Train AUROC: 0.8470 Validation Loss: 0.4403 Validation AUROC: 0.8711 Accuracy 81 %
EPOCH 7 Training Loss: 0.3920 Train AUROC: 0.8714 Validation Loss: 0.3588 Validation AUROC: 0.8846 Accuracy 85 %


/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


EPOCH 8 Training Loss: 0.3368 Train AUROC: 0.9043 Validation Loss: 0.4838 Validation AUROC: 0.8717 Accuracy 77 %
EPOCH 9 Training Loss: 0.3568 Train AUROC: 0.8911 Validation Loss: 0.3857 Validation AUROC: 0.8800 Accuracy 80 %
EPOCH 10 Training Loss: 0.3320 Train AUROC: 0.9085 Validation Loss: 0.3970 Validation AUROC: 0.8912 Accuracy 80 %
EPOCH 11 Training Loss: 0.3165 Train AUROC: 0.9168 Validation Loss: 0.4008 Validation AUROC: 0.8856 Accuracy 82 %
EPOCH 12 Training Loss: 0.3069 Train AUROC: 0.9235 Validation Loss: 0.3725 Validation AUROC: 0.8980 Accuracy 83 %
EPOCH 13 Training Loss: 0.3236 Train AUROC: 0.9165 Validation Loss: 0.3778 Validation AUROC: 0.9178 Accuracy 81 %
EPOCH 14 Training Loss: 0.3032 Train AUROC: 0.9273 Validation Loss: 0.3405 Validation AUROC: 0.9025 Accuracy 86 %
EPOCH 15 Training Loss: 0.3080 Train AUROC: 0.9247 Validation Loss: 0.3650 Validation AUROC: 0.8919 Accuracy 82 %
EPOCH 16 Training Loss: 0.3013 Train AUROC: 0.9287 Validation Loss: 0.3316 Validation AURO

In [ ]:
results = [result.item() for result in results] 

In [ ]:
print('Mean:',"{:.3f}".format(mean(results)*100),'\nStandard deviation:', "{:.3f}".format(stdev(results)*100))